# Gallery Example: Erl/Erl/1 Reentrant Queue

This example demonstrates an Erlang/Erlang/1 queueing system with reentrant routing:
- **Arrivals**: Erlang distribution
- **Service**: Erlang distribution (Class1), Exponential (Class2)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs may switch class and pass through the queue again

This model includes probabilistic class switching.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_erlerl1_reentrant(n=5): Network {
    """Create Erl/Erl/1-Reentrant queueing model"""
    val model = Network("Erl/Erl/1-Reentrant")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// Erlang arrival for Class1, Class2 is disabled at source
    source.setArrival(oclass1, Erlang.fit_mean_and_order(1, n))
    source.setArrival(oclass2, Disabled())
// Erlang service for Class1, Exponential for Class2
    queue.setService(oclass1, Erlang.fit_mean_and_order(0.5, n))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with probabilistic class switching
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)        # Arrivals go to queue as Class1
    P.addClassSwitch(oclass1, oclass2, queue, queue, 0.5)  # 50% switch to Class2
    P.addRoute(oclass2, queue, sink, 0.5)          # 50% of Class2 exits
    model.link(P)
    
    return model
// Create the model
val model = gallery_erlerl1_reentrant()

## About This Model

This reentrant model features:
- Both Erlang arrivals and Erlang service (low variability)
- Probabilistic class switching: 50% chance to switch from Class1 to Class2
- Different service distributions by class

The low variability of Erlang distributions (SCV = 1/n) combined with reentrant routing creates interesting dynamics in the queue.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)